<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" align="right">

<h1 align="right">
M3.2 – تحليل مكعب بيانات الهطول العالمي
</h1>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
جزء من:
<span dir="ltr" style="unicode-bidi: isolate;">
  <a href="https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources" dir="ltr">
    Open Science for Water Resources
  </a>
</span>
</p>

</p>

</div>


</div>

In [ ]:
import datetime
import glob
import earthaccess
import numpy as np
import h5py
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تتكوّن دورة المياه البرّية من <strong>مخزونات</strong> و<strong>تدفّقات</strong> للمياه. تشمل المخزونات أشكال التخزين طويلة الأمد مثل البحيرات والخزانات والمياه الجوفية وبخار الماء في الغلاف الجوي. أمّا التدفّقات فتشير إلى أي انتقال للمياه من مخزون إلى آخر.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تمامًا كما في الحسابات البنكية، يمكننا دراسة التدفّقات لفهم الصورة العامة. فإذا لاحظنا أن المال يغادر حسابًا بكثرة، يمكننا الاستنتاج بأن رصيد الحساب قد يكون منخفضًا. ويمكننا إعداد ميزانية لحساب التدفقات الداخلة والخارجة لتحديد ما إذا كان الرصيد يزداد أم يتناقص.
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
لدراسة توافر المياه في حوض أو مستجمع مائي، ننظر إلى ميزانية المياه التالية:
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: center;">
<span dir="ltr" style="unicode-bidi: isolate;">
$$
P = E + R + \Delta S
$$
</span>
</p>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
حيث:
</p>

<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li>
<span dir="ltr" style="unicode-bidi: isolate;">P</span>
هو الهطول، ويُفترض أنه المصدر الأساسي لدخول المياه إلى الحوض.
</li>

<li>
<span dir="ltr" style="unicode-bidi: isolate;">E</span>
هو <strong>النتح–التبخر</strong>
<span dir="ltr" style="unicode-bidi: isolate;">(evapotranspiration)</span>،
أي مجموع المياه التي تتبخر أو تمتصها النباتات من التربة عبر عملية النتح.
</li>

<li>
<span dir="ltr" style="unicode-bidi: isolate;">R</span>
هو الجريان السطحي، أي المياه التي تغادر الحوض عبر التدفق فوق السطح، وغالبًا عبر نهر رئيسي.
</li>

<li>
<span dir="ltr" style="unicode-bidi: isolate;">\Delta S</span>
هو التغير في التخزين، ويشير عادةً إلى التغير في كمية المياه المتاحة، وخاصة المياه الجوفية.
</li>
</ul>

<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
تكمن فكرة ميزانية المياه في أننا، بعد احتساب المدخلات
(<span dir="ltr" style="unicode-bidi: isolate;">P</span>)
والمخرجات
(<span dir="ltr" style="unicode-bidi: isolate;">E</span>
و<span dir="ltr" style="unicode-bidi: isolate;">R</span>)،
يمكننا تقدير مقدار زيادة أو نقصان مخزون المياه المتاحة في الحوض
(<span dir="ltr" style="unicode-bidi: isolate;">\Delta S</span>).
</p>

</div>


</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">فيما يلي توضيح لهذه التدفقات المائية داخل مستجمع مياه.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><img src="./assets/water_budget.png" alt="water budget" dir="ltr"></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://www.usgs.gov/media/images/components-a-simple-water-budget-part-a-watershed" dir="ltr">الصورة بإذن من USGS</a></p>
</div>

</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إذًا، لحساب ميزانية مائية نحتاج إلى ثلاثة عناصر على الأقل: بيانات الهطول، وبيانات الجريان السطحي، وبيانات النتح-التبخر.</p>
</div>

</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">البيانات تنزيل</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <a href="https://doi.org/10.5281/zenodo.16896169" dir="ltr">ملفات Shapefiles المستخدمة في هذا الدرس يمكن تنزيلها من هذا الرابط</a></p>
</div>

</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الهطول على مستوى الحوض حساب</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الحوض الذي سنعتمده في هذه الدراسة هو حوض نهر يلوستون. يتم قياس جريان هذا الحوض عند نهر يلوستون بالقرب من سيدني، مونتانا <span dir="ltr" style="unicode-bidi: isolate;">(U.S.A.)</span>.</p>
</div>

</div>

In [ ]:
import geopandas

basin = geopandas.read_file('data/shp/YellowstoneRiver_drainage_WSG84.shp')
river = geopandas.read_file('data/shp/YellowstoneRiver_course_WSG84.shp')
states = geopandas.read_file('data/shp/YellowstoneRiver_states_WGS84.shp')
basin

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">فيما يلي رسم لمنطقة الدراسة يوضح مساحة التصريف <span dir="ltr" style="unicode-bidi: isolate;">(basin)</span> لنهر يلوستون. نقطة خروج النهر من الحوض تمثل محطة القياس الخاصة بنا في شرق مونتانا.</p>
</div>

</div>

In [ ]:
ax = states.plot(edgecolor = 'black', color = 'lightgray')
basin.plot(ax = ax, edgecolor = 'darkblue', color = 'none')
river.plot(ax = ax, edgecolor = 'green', label = 'Yellowstone River')
pyplot.legend(loc = 'lower left')
pyplot.show()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> بيانات الهطول تنزيل</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لحساب كمية الهطول التي تدخل الحوض، سنستخدم مجموعة بيانات <span dir="ltr" style="unicode-bidi: isolate;">NASA</span> المسماة <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> والتي شُرحت بالتفصيل في درس سابق. سنستخدم تحديدًا نسخة شهرية من <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> تقدّر معدل الهطول على شبكة عالمية.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://github.com/OpenClimateScience/M1-Open-Climate-Data/blob/main/notebooks/05_Earth_Observation_Data.ipynb" dir="ltr">وصف IMERG-Final في درس سابق</a></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <a href="https://dx.doi.org/10.5067/GPM/IMERG/3B-MONTH/07" dir="ltr">وثائق IMERG-Final الشهرية</a></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لننزّل 10 سنوات من بيانات الهطول الشهرية، من <span dir="ltr" style="unicode-bidi: isolate;">2014</span> حتى <span dir="ltr" style="unicode-bidi: isolate;">2023</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>ملاحظة</strong> يجب أولًا إنشاء المجلد الذي تريد وضع البيانات فيه. هنا نفترض أنك تريد وضعها في مجلد باسم <span dir="ltr" style="unicode-bidi: isolate;">data/IMERG-Final_monthly</span>.</p>
</div>

</div>

In [ ]:
results = earthaccess.search_data(
    short_name = 'GPM_3IMERGM',
    temporal = ('2014-01-01', '2023-12-31'))

In [ ]:
earthaccess.download(results, 'data/IMERG-Final_monthly')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">HDF5</span> ملفات متعددة التعامل مع</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بيانات <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> تأتي بصيغة <span dir="ltr" style="unicode-bidi: isolate;">HDF5</span>، وقد يكون التعامل معها صعبًا عندما نهتم بالإحداثيات المكانية. مكتبتنا المفضلة <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لا تعرف تلقائيًا كيفية تفسير نظم الإحداثيات في ملفات <span dir="ltr" style="unicode-bidi: isolate;">HDF5</span>، لذلك سنؤكد بعض التفاصيل الأساسية بأنفسنا بعد فتح أحد المقاطع <span dir="ltr" style="unicode-bidi: isolate;">(granules)</span> باستخدام مكتبة <span dir="ltr" style="unicode-bidi: isolate;">h5py</span>.</p>
</div>

</div>

In [ ]:
with h5py.File('data/IMERG-Final_monthly/3B-MO.MS.MRG.3IMERG.20180701-S000000-E235959.07.V07B.HDF5', 'r') as hdf:
    longitude = hdf['Grid/lon'][:]
    latitude = hdf['Grid/lat'][:]
    print(longitude.shape)
    print(latitude.shape)
    print(hdf['Grid/precipitation'].shape)
    print(hdf['Grid/precipitation'].attrs['units'])

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يبدو أن بياناتنا على شبكة طول-عرض، تضم <span dir="ltr" style="unicode-bidi: isolate;">3600</span> عمودًا و<span dir="ltr" style="unicode-bidi: isolate;">1800</span> صفًا. وهذا يعادل دقة مكانية قدرها <span dir="ltr" style="unicode-bidi: isolate;">0.1</span> درجة. وحدات الهطول هي مليمتر في الساعة <span dir="ltr" style="unicode-bidi: isolate;">(mm/hr)</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لدينا 10 سنوات من البيانات الشهرية، لذا يجب أن يكون لدينا <span dir="ltr" style="unicode-bidi: isolate;">120</span> ملفًا.</p>
</div>

</div>

In [ ]:
file_list = glob.glob('data/IMERG-Final_monthly/*.HDF5')
file_list.sort()
len(file_list)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن استخراج معلومات التاريخ الخاصة بكل ملف من اسم الملف.</p>
</div>

</div>

In [ ]:
file_list[0].split('.')[4][0:8]

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن دعنا نرى كيف يمكن فتح هذه البيانات باستخدام <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>.</p><h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">xarray</span> مع <span dir="ltr" style="unicode-bidi: isolate;">HDF5</span> ملفات قراءة</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مرة أخرى، لأن الملف بصيغة <span dir="ltr" style="unicode-bidi: isolate;">HDF5</span>، فمن الصعب على <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> تفسير نظم الإحداثيات وبنية الملف. لذلك عند استخدام <span dir="ltr" style="unicode-bidi: isolate;">xr.open_dataset()</span> نحتاج إلى ما يلي.</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تحديد أن <span dir="ltr" style="unicode-bidi: isolate;">group</span> الذي تُخزَّن فيه مجموعات البيانات اسمه <span dir="ltr" style="unicode-bidi: isolate;">&#x27;Grid&#x27;</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إخبار <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> بعدم محاولة تفسير معلومات التاريخ/الوقت: <span dir="ltr" style="unicode-bidi: isolate;">decode_times = False</span>.</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نحن مهتمون فقط بمتغير <span dir="ltr" style="unicode-bidi: isolate;">&#x27;precipitation&#x27;</span>، لذلك نستخدم الدالة <span dir="ltr" style="unicode-bidi: isolate;">get()</span> لاختيار هذا المتغير فقط من مجموعة بيانات <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وبسبب محدودية معرفة <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> بتفاصيل <span dir="ltr" style="unicode-bidi: isolate;">HDF5</span> نحتاج إلى استخدام <span dir="ltr" style="unicode-bidi: isolate;">assign_coords()</span> لتعريف إحداثيات البيانات.</p>
</div>

</div>

In [ ]:
filename = file_list[0]

# We only care about the "precipitation" variable, but we want an xarray.Dataset,
#    so we include the name of the variable(s) we want as a list in get()
ds = xr.open_dataset(filename, group = 'Grid', decode_times = False).get(['precipitation'])

# Define the missing coordinates
date = datetime.datetime.strptime(filename.split('.')[4][0:8], '%Y%m%d') # e.g., "20180101"
ds = ds.assign_coords({
    'time': [date], 'x': longitude, 'y': latitude
})

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أخيرًا أصبحنا جاهزين لعرض البيانات. لإبراز بعض الأنماط، سنضبط <span dir="ltr" style="unicode-bidi: isolate;">vmax = 1</span> بحيث يمتد شريط الألوان حتى <span dir="ltr" style="unicode-bidi: isolate;">1</span> <span dir="ltr" style="unicode-bidi: isolate;">mm/hr</span> كحد أعلى.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قد يبدو الرسم غريبًا لأن منتج <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> يقدّر الهطول فوق اليابسة والمحيطات معًا. مع ذلك، ينبغي أن تتمكن من تمييز غرب أوروبا وأيسلندا في أعلى منتصف الصورة. ولاحظ أيضًا أن المناطق المدارية، كما هو متوقع، تتلقى هطولًا أكبر من بقية العالم.</p>
</div>

</div>

In [ ]:
ds.precipitation.plot(x = 'lon', vmax = 1)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أنظمة الإسناد المكاني</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">البيانات المكانية تختلف عن غيرها. فإضافة إلى قيم البيانات نفسها (مثل الهطول)، ترتبط كل قيمة بإحداثيات مكانية تصف موقعًا على سطح الأرض.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لأن مجموعات البيانات لدينا غالبًا ثنائية الأبعاد ومسطحة (مثل بيانات الهطول أعلاه)، نحتاج إلى طريقة لربط كل بكسل في التمثيل المسطح بموقعه على الأرض الكروية. يصف <strong>نظام الإسناد المرجعي للإحداثيات</strong> <span dir="ltr" style="unicode-bidi: isolate;">(CRS)</span> كيفية ارتباط التمثيل المسطح بسطح الأرض.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">غالبًا ما يُوصف <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> بواسطة معرف فريد يسمى رمز <span dir="ltr" style="unicode-bidi: isolate;">EPSG</span>، حيث يشير <span dir="ltr" style="unicode-bidi: isolate;">EPSG</span> إلى European Petroleum Survey Group.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 يمكنك التعرف أكثر على رموز <span dir="ltr" style="unicode-bidi: isolate;">EPSG</span> والبحث عن رمز نظام محدد عبر موقع <a href="https://epsg.io/" dir="ltr">epsg.io</a>.</p>
</div>

</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">xarray</span> Dataset تقطيع مكاني</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">هذه مجموعة بيانات عالمية، لكننا مهتمون فقط بالهطول داخل حوض نهر يلوستون. لقد رأينا سابقًا كيف نقتطع مجموعة بيانات <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> إلى نقطة معينة أو ضمن صندوق إحاطة مستطيل. لكن كيف نقتطع البيانات وفق شكل غير منتظم مثل حوض تصريف</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>أولًا</strong> نحتاج إلى تعريف نظام الإسناد المكاني لبياناتنا. يمكننا ذلك عبر الوصول إلى خاصية <span dir="ltr" style="unicode-bidi: isolate;">rio</span> في مجموعة البيانات.</p><h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">انتبه</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>لكي يعمل المثال التالي يجب أن تكون وحدة</strong> <span dir="ltr" style="unicode-bidi: isolate;">rioxarray</span> <strong>مثبتة</strong>. لا يلزم استيرادها صراحة، لكن يجب تثبيتها.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يتم الوصول إلى خاصية <span dir="ltr" style="unicode-bidi: isolate;">rio</span> عبر <span dir="ltr" style="unicode-bidi: isolate;">ds.rio</span> إذا كان اسم مجموعة البيانات هو <span dir="ltr" style="unicode-bidi: isolate;">ds</span>. <span dir="ltr" style="unicode-bidi: isolate;">ds.rio</span> يمنحنا أدوات متعددة للتعامل مع الإحداثيات والسمات المكانية. ويسمى كذلك لأنه مشتق من مكتبة <span dir="ltr" style="unicode-bidi: isolate;">rasterio</span> التي تختصر عادة إلى <span dir="ltr" style="unicode-bidi: isolate;">rio</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لنعرف <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> لبياناتنا باستخدام رمز <span dir="ltr" style="unicode-bidi: isolate;">EPSG</span>. بالنسبة لبيانات الطول-العرض يمكننا عادة افتراض أن أفضل وصف هو نظام الإحداثيات الجغرافي <span dir="ltr" style="unicode-bidi: isolate;">WGS84</span> الذي يحمل الرمز <span dir="ltr" style="unicode-bidi: isolate;">4326</span>.</p>
</div>

</div>

In [ ]:
from pyproj import CRS

# NOTE: We must have rioxarray installed to be able to access the 
#  "rio" property of xarray Datasets; here, we define the CRS of this
#  dataset, which was previously undefined

# Tell xarray that this dataset's CRS is the WGS84 Geographic Coordinate System
ds = ds.rio.write_crs(CRS.from_epsg(4326))

# Also, we need to indicate which of the coordinates correspond to X and Y dimensions
ds = ds.rio.set_spatial_dims(x_dim = 'lon', y_dim = 'lat')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد تعريف <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> والإحداثيات المكانية، يصبح من السهل قص شبكة الهطول ضمن حدود الحوض.</p><h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">انتبه</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لكي يعمل ذلك، يجب أن يكون <span dir="ltr" style="unicode-bidi: isolate;">CRS</span> لبياناتنا الشبكية <span dir="ltr" style="unicode-bidi: isolate;">ds</span> مطابقًا لـ<span dir="ltr" style="unicode-bidi: isolate;">CRS</span> الخاص بالحوض <span dir="ltr" style="unicode-bidi: isolate;">basin</span>. يمكننا التحقق من ذلك عبر:</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">ds.rio.crs == basin.crs</span></p>
</div>

</div>

In [ ]:
ds_clip = ds.rio.clip(basin.geometry.values)
ds_clip.precipitation

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا ملاحظة أن مجموعة البيانات الجديدة <span dir="ltr" style="unicode-bidi: isolate;">ds_clip</span> أصغر حجمًا. لنتحقق أكثر عبر رسمها.</p>
</div>

</div>

In [ ]:
ds_clip.precipitation[0].plot(x = 'lon')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">معالجة بيانات خط أنابيب إنشاء</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نجحت هذه الخطوات مع مجموعة بيانات واحدة، لكن لدينا عدة ملفات لمعالجتها. لننشئ حلقة <span dir="ltr" style="unicode-bidi: isolate;">for</span> للتعامل معها.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن استخدام الدالة <span dir="ltr" style="unicode-bidi: isolate;">concat()</span> في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لدمج عدة مجموعات بيانات في مجموعة واحدة. هنا، لأن كل مقطع <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> يمثل تاريخًا مختلفًا (شهرًا)، سنجمعها على محور <span dir="ltr" style="unicode-bidi: isolate;">time</span>.</p>
</div>

</div>

In [ ]:
datasets = []

for filename in file_list:
    date = datetime.datetime.strptime(filename.split('.')[4][0:8], '%Y%m%d')
    ds0 = xr.open_dataset(filename, group = 'Grid', decode_times = False).get(['precipitation'])
    ds0 = ds0.assign_coords({
        'time': [date], 'x': longitude, 'y': latitude
    })

    # Tell xarray that this dataset's CRS is the WGS84 Geographic Coordinate System
    ds0 = ds0.rio.write_crs(CRS.from_epsg(4326))
    
    # Also, we need to indicate which of the coordinates correspond to X and Y dimensions
    ds0 = ds0.rio.set_spatial_dims(x_dim = 'lon', y_dim = 'lat')

    ds_clip = ds0.rio.clip(basin.geometry.values)
    
    # Only write the file if it doesn't exist (in case we run this again)
    datasets.append(ds_clip)

# Combine the datasets together along the "time" axis
ds = xr.concat(datasets, dim = 'time')
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">معدل الهطول الإجمالي على مستوى الحوض حساب</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد ذلك، نحتاج إلى تحويل وحدات القياس. لدينا حاليًا الهطول بوحدة <span dir="ltr" style="unicode-bidi: isolate;">mm/hr</span> بينما البيانات شهرية. للتحويل من <span dir="ltr" style="unicode-bidi: isolate;">mm/hr</span> إلى <span dir="ltr" style="unicode-bidi: isolate;">mm/month</span> نحتاج إلى معرفة عدد الأيام في كل شهر.</p>
</div>

</div>

In [ ]:
import calendar

calendar.mdays

In [ ]:
days_in_month = np.array(calendar.mdays)[ds.coords['time.month'].values]
days_in_month

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يبدو من السهل ضرب بياناتنا الشهرية (معدل بالساعة) في عدد الأيام (ومعامل <span dir="ltr" style="unicode-bidi: isolate;">24</span> ساعة). لكن عند العمل مع بيانات مصفوفات يجب الانتباه إلى شكل <span dir="ltr" style="unicode-bidi: isolate;">(shape)</span> البيانات.</p>
</div>

</div>

In [ ]:
ds.precipitation.shape

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تُخزَّن بيانات الهطول لدينا في مصفوفة <span dir="ltr" style="unicode-bidi: isolate;">NumPy</span> بثلاثة محاور.</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">120</span> شهرًا</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">72</span> صفًا (كل منها <span dir="ltr" style="unicode-bidi: isolate;">0.1</span> درجة عرض)</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">53</span> عمودًا (كل منها <span dir="ltr" style="unicode-bidi: isolate;">0.1</span> درجة طول)</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عند ضرب المصفوفات، ينبغي أن تكون الأشكال متوافقة. لذلك نحتاج أيضًا إلى إعادة تشكيل مصفوفة <span dir="ltr" style="unicode-bidi: isolate;">days_in_month</span> لتتطابق مع شكل بيانات الهطول.</p>
</div>

</div>

In [ ]:
# Converting from [mm hour-1] to [mm month-1]
ds['precip_monthly'] = ds.precipitation * 24 * days_in_month.reshape((days_in_month.size, 1, 1))
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أخيرًا أصبحنا جاهزين لحساب الهطول الكلي للحوض.</strong></p><h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الهطول الكلي حساب</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قد تميل إلى جمع قيم جميع البكسلات داخل الحوض لتحديد إجمالي الهطول في شهر معين. لكن ذلك سيبالغ كثيرًا في التقدير، لأن وحدات القياس (<span dir="ltr" style="unicode-bidi: isolate;">mm/hr</span> أو <span dir="ltr" style="unicode-bidi: isolate;">mm/month</span>) مرتبطة بمساحة مكانية محددة. أي أن ارتفاع عمود الماء (بالمليمتر) يتوافق مع مساحة ثابتة. إذا كانت المساحة أكبر فإن ارتفاع الماء ينخفض لأن الماء ينتشر ليغطي مساحة أكبر. <strong>ما نحتاجه فعليًا هو حساب المتوسط</strong> لقيم الهطول داخل الحوض.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">طريقة أخرى للتفكير: تخيّل خزانًا فيه حاجز يقسمه إلى منطقتين متساويتين. في اليسار عمق الماء <span dir="ltr" style="unicode-bidi: isolate;">4</span> مم، وفي اليمين <span dir="ltr" style="unicode-bidi: isolate;">2</span> مم. إذا أزلنا الحاجز فسيستقر الماء عند <span dir="ltr" style="unicode-bidi: isolate;">3</span> مم، وهو متوسط <span dir="ltr" style="unicode-bidi: isolate;">2</span> و<span dir="ltr" style="unicode-bidi: isolate;">4</span> وليس مجموعهما.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 عند حساب المتوسط على كامل الحوض في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> نحدد أننا نريد طي محوري <span dir="ltr" style="unicode-bidi: isolate;">&#x27;lon&#x27;</span> و<span dir="ltr" style="unicode-bidi: isolate;">&#x27;lat&#x27;</span> لنحصل في النهاية على محور <span dir="ltr" style="unicode-bidi: isolate;">&#x27;time&#x27;</span> فقط.</p>
</div>

</div>

In [ ]:
# Compute the mean over all longitude-latitude bins
precip_series = ds.precip_monthly.mean(['lon','lat']).values

# Plot the result
pyplot.plot(precip_series)
pyplot.ylabel('Basin-wide precipitation (mm per month)')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن نحن جاهزون لحفظ مجموعة البيانات لاستخدامها لاحقًا. سنحفظها في ملف <span dir="ltr" style="unicode-bidi: isolate;">netCDF4</span>.</p>
</div>

</div>

In [ ]:
ds.to_netcdf('./processed/IMERG-Final_precip_monthly_2014-2023.nc')